# TPM - Atividade III

Crie uma tabela com os valores das posições de cada um dos portfólios desenvolvidos, e dê os dados da posição de cada um dos fundos na data de 30/03/2023, calcule o percentual de concentração de cada um dos portfólios e a porcentagem do ADTV dos últimos 30 dias que cada posição possui.

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import config as cfg
import helpers as hlp

In [2]:
# Lendo a lista de portfólios
df_portfolio_list = pd.read_csv(f'{cfg.PATH_RESULTADOS}/portfolio_files.csv')

df_portfolio_list.head()

,Portfolio,File Name
0,Blue Chips,blue_chips.parquet
1,Mid Caps,mid_caps.parquet
2,Small Caps,small_caps.parquet
3,Micro Caps,micro_caps.parquet
4,High Return,high_return.parquet


In [3]:
# Lendo a composição dos portfolios
df_portfolios_asset = pd.read_parquet(cfg.PATH_PORTFOLIO_PARQUET)

In [4]:
# Lendo o histórico do volume de negociações
df_volume = pd.read_parquet(cfg.PATH_VOLUME_PARQUET)
df_volume.set_index('Data', inplace=True)

end_date = pd.to_datetime('2023-03-30')
start_date = pd.to_datetime(end_date) - pd.DateOffset(days=30)

print(f'start_date => {start_date}')
print(f'end_date => {end_date}')

mask = (df_volume.index >= start_date) & (df_volume.index <= end_date)
df_volume = df_volume.loc[mask]


start_date => 2023-02-28 00:00:00
end_date => 2023-03-30 00:00:00


In [5]:
# Recuperando a quantidade de ações dispóníveis para negociação
df_share_outstanding = pd.read_parquet(cfg.PATH_SHARESOUTSTANDING_PARQUET)

## 1. Calculando para o portfólio de Blue Ships

In [6]:
# Recuperando o arquivo
portfolio_name = df_portfolio_list['Portfolio'].iloc[0]
file_name = df_portfolio_list['File Name'].iloc[0]

df_blue_chips = pd.read_parquet(f"{cfg.PATH_CALCULATED_PORTFOLIO_DIR}/{file_name}")

In [7]:
# Recuperando a posição em 30/03/2023
posicao_data_base = df_blue_chips.loc['2023-03-30']

# Recuperando a lista dos ativos do portfloio
stock_list = df_portfolios_asset[portfolio_name].to_list()

In [8]:
# Calculando a concentração do portfolio
concentration_list = []

for stock in tqdm(stock_list):
    mean_volume = df_volume[stock].mean()
    shares_outstanding = df_share_outstanding[df_share_outstanding['Stock'] == stock]['SharesOutstanding'].values[0]

    position = {
        'Ativo': stock,
        'Qtd': int(posicao_data_base[f'{stock}_qtd']),
        'Preco': posicao_data_base[f'{stock}_price'],
        'Valor da posição': posicao_data_base[stock],
        'Concentração da posição (%)': posicao_data_base[stock]/posicao_data_base['TOTAL AMOUNT'] * 100,
        'ADTV 30D (%)': mean_volume / shares_outstanding * 100
    }
    concentration_list.append(position)

df_concentration = pd.DataFrame(concentration_list)

df_concentration.head()

  0%|          | 0/15 [00:00<?, ?it/s]

,Ativo,Qtd,Preco,Valor da posição,Concentração da posição (%),ADTV 30D (%)
0,VALE3.SA,17169,81.82,1404767.58,11.967548,41.473876
1,PETR4.SA,44683,23.97,1071051.51,9.124542,29.369777
2,ITUB4.SA,24520,24.62,603682.40,5.142914,18.244404
3,WEGE3.SA,40207,41.86,1683065.02,14.338430,5.940912
4,BBDC4.SA,28122,13.38,376272.36,3.205553,12.286943


## 2. Calculando para todos os portfolios.

In [9]:
def calculate_portfolio_concentration(df, portfolio_name, base_date):
    # Recuperando a lista dos ativos do portfloio
    stock_list = df_portfolios_asset[portfolio_name].to_list()  

    base_position = df.loc[base_date]
    
    # Calculando a concentração do portfolio
    concentration_list = []

    for stock in stock_list:
        mean_volume = df_volume[stock].mean()
        shares_outstanding = df_share_outstanding[df_share_outstanding['Stock'] == stock]['SharesOutstanding'].values[0]

        position = {
            'Porfolio': portfolio_name,
            'Ativo': stock,
            'Qtd': int(base_position[f'{stock}_qtd']),
            'Preco': base_position[f'{stock}_price'],
            'Valor da posição': base_position[stock],
            'Concentração da posição (%)': base_position[stock]/base_position['TOTAL AMOUNT'] * 100,
            'ADTV 30D (%)': mean_volume / shares_outstanding * 100
        }
        concentration_list.append(position)

    df_concentration = pd.DataFrame(concentration_list)

    return df_concentration


In [10]:
base_date = '2023-03-30'

df_concentration_list = []

#Acrescentando o retorno dos portfólios
for index, row in tqdm(df_portfolio_list.iterrows(), total=len(df_portfolio_list)):
    portfolio_name = row['Portfolio']
    file_name = row['File Name']

    df_portfolio = pd.read_parquet(f"{cfg.PATH_CALCULATED_PORTFOLIO_DIR}/{file_name}")

    df_concentration_portfolio = calculate_portfolio_concentration(df_portfolio, portfolio_name, base_date)

    df_concentration_list.append(df_concentration_portfolio)
    

df_concentration = pd.concat(df_concentration_list, axis=0)
df_concentration.head()

  0%|          | 0/24 [00:00<?, ?it/s]

,Porfolio,Ativo,Qtd,Preco,Valor da posição,Concentração da posição (%),ADTV 30D (%)
0,Blue Chips,VALE3.SA,17169,81.82,1404767.58,11.967548,41.473876
1,Blue Chips,PETR4.SA,44683,23.97,1071051.51,9.124542,29.369777
2,Blue Chips,ITUB4.SA,24520,24.62,603682.40,5.142914,18.244404
3,Blue Chips,WEGE3.SA,40207,41.86,1683065.02,14.338430,5.940912
4,Blue Chips,BBDC4.SA,28122,13.38,376272.36,3.205553,12.286943


In [11]:
hlp.save_to_parquet(df_concentration, cfg.PATH_TABELA_CONCENTRACAO_PORTFOLIO)